In [1]:
import requests
import json
import numpy as np
import pandas as pd
from pandas import DataFrame as df

In [2]:
fips_code = pd.read_csv("unique_county.csv")
fips_code.head(3)

,state_code,state_name,county_code,county_name,KEY,FIPS
0,6,CA,37,LOS ANGELES,CA LOS ANGELES,6037
1,6,CA,53,MONTEREY,CA MONTEREY,6053
2,6,CA,71,SAN BERNARDINO,CA SAN BERNARDINO,6071


In [3]:
fips_code['series_ID'] = 'LAUCN0' + fips_code['FIPS'].astype('str') +'0000000003'
#fips_code['series_ID'] = 'LAUCN' + fips_code['FIPS'].astype('str') +'0000000003'

total_seriesID = list(fips_code['series_ID'])
fips_code.head(3)

,state_code,state_name,county_code,county_name,KEY,FIPS,series_ID
0,6,CA,37,LOS ANGELES,CA LOS ANGELES,6037,LAUCN060370000000003
1,6,CA,53,MONTEREY,CA MONTEREY,6053,LAUCN060530000000003
2,6,CA,71,SAN BERNARDINO,CA SAN BERNARDINO,6071,LAUCN060710000000003


In [4]:
len(total_seriesID)

181

In [5]:
#generate fixed master_df
master_df = pd.DataFrame(columns = ["county", "year", "month", "value"])
master_df

,county,year,month,value


In [ ]:
# seriesid에 25개까지 county 함께 담아서 보내면, 한번 보내는게 한번 쿼리보내는 거임 
# total_seriesID 적당히 쪼개서 쿼리 제한 안받게 보내면 됨

for component in total_seriesID:
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": [component],"startyear":"2004", "endyear":"2007"})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    
    series = json_data['Results']['series'][0]
    series_id = series['seriesID'][6:-10]
    #series_id = series['seriesID'][5:-10]

    for item in series['data']:
        year = item['year']
        month = item['period'][1:]
        value = item['value']
        df1 = pd.DataFrame(data = [[series_id, year, month, value]], columns = ["county", "year", "month", "value"])
        master_df = master_df.append(df1)
    